This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [1]:
# 05ex - 1

import pandas as pd

file = "C://Users/MZ/desktop/data_000637.txt"
df = pd.read_csv(file)
print(df.tail(10))      # reading the last N rows. N=10 here

         HEAD  FPGA  TDC_CHANNEL   ORBIT_CNT  BX_COUNTER  TDC_MEAS
1310710     1     1           36  3869211171         427        13
1310711     1     1           39  3869211171         430         0
1310712     1     1           35  3869211171         429         4
1310713     1     0           64  3869211171         758        18
1310714     1     0           60  3869211171         762         2
1310715     1     0           62  3869211171         762        14
1310716     1     1            4  3869211171         763        11
1310717     1     0           64  3869211171         764         0
1310718     1     0          139  3869211171         769         0
1310719     1     0           61  3869211171         762        18


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [ ]:
# 05ex - 2

import pandas as pd
file = "C://Users/MZ/desktop/data_000637.txt"
df = pd.read_csv(file)

print(df.loc[1310718][4])

3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [6]:
# 05ex - 3

x_1 = df['TDC_MEAS']
x_2 = df['BX_COUNTER']
x_3 = df['ORBIT_CNT']
y_1 = x_1.sum()
y_2 = x_2.sum() + y_1 * 30
y_3 = x_3.sum() + y_2 * 3563
hour = y_3//(1000000000 * 3600)
minute = (y_3 - (hour * 3600 * 1000000000)) // (60 * 1000000000)
second = (y_3 - (hour * 3600 * 1000000000) - (minute * 60 * 1000000000)) // 1000000000

print(hour, minute, second)

1411 33 50


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [3]:
# 05ex - 5

import pandas as pd

file = "C://Users/MZ/desktop/data_000637.txt"
df = pd.read_csv(file)
print(df.groupby('TDC_CHANNEL').count().sort_values(by = "HEAD", ascending = False).head(3))

               HEAD    FPGA  ORBIT_CNT  BX_COUNTER  TDC_MEAS
TDC_CHANNEL                                                 
139          108059  108059     108059      108059    108059
64            66020   66020      66020       66020     66020
63            64642   64642      64642       64642     64642


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [11]:
# 05ex - 6

x = df.groupby("ORBIT_CNT")["ORBIT_CNT"].count()
print(x.count())
print("\n")
m = df[df['TDC_CHANNEL'] == 139]
y = m.groupby("ORBIT_CNT")["ORBIT_CNT"].count()
print(y.count())

11001


10976


7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel